## Jupyter Notebooks Intro

Jupyter notebooks allow for the execution of independent snippets of Python code in "cells". These cells can be executed multiple times and the notebook acts as though each line of Python code has been executed in sequence -- as though the sequence of lines executed were in one large Python script.

It also allows for the combination of code cells with markdown text cells (_like this one!_).


Here are a couple of tips for working in the notebook:

  - to navigate between cells: Esc and up/down arrows, Enter to start editing a cell
  - execute cell and move to next cell: shift + Enter
  - execute cell and stay in that cell: ctrl + Enter
  - add cell below the current cell: Esc then "b"
  - add cell above the current cell: Esc then "a"
  








In [ ]:
x = 1.0
print(x)

This is the OpenMC Workshop.

In [ ]:
arr = [1.0, 2.0]

In [ ]:
arr

In [ ]:
l = [1, 2, 3, 4, 5]
print(l[2:])
print(l[:-2])

In [ ]:
import math

In [ ]:
math.cos(math.pi)

In [ ]:
import openmc

# Model Components

  - Materials
  - Geometry
  - Settings
  - Tallies (optional)
  - Plots (optional)

## Defining Materials

In [ ]:
steel = openmc.Material(name="steel")

In [ ]:
steel.add_nuclide("Fe56", 1.0)
steel.add_nuclide("Fe57", 0.5)
steel.set_density('g/cc', 5.0)

In [ ]:
print(steel)

In [ ]:
openmc.Material()

In [ ]:
# steel.volume = 'abc'

In [ ]:
steel.get_nuclide_atom_densities()

In [ ]:
air = openmc.Material()
air.add_nuclide('N14', 1.0)
air.set_density('g/cc', 0.001)

In [ ]:
air

In [ ]:
materials = openmc.Materials([steel, air])

In [ ]:
materials.export_to_xml()

In [ ]:
!cat materials.xml

In [ ]:
zr = openmc.Material()
zr.add_element('Zr', 1.0)
zr.set_density('g/cc', 6.0)

In [ ]:
zr

<iframe src="https://playground.tensorflow.org" width="1200" height="1000"></iframe>


<div class="alert alert-block alert-info">
    <h3>Setting OpenMC cross section data</h3>
    Resources:
    <ul>  
      <li><a href="https://docs.openmc.org/en/stable/usersguide/cross_sections.html">Documentation on cross section configuration</a></li>
    <li><a href="https://openmc.org/official-data-libraries/">Pre-processed nuclear data libraries</a></li>
    </ul>
</div>

In [ ]:
!echo $OPENMC_CROSS_SECTIONS

In [ ]:
!head -n 50 $OPENMC_CROSS_SECTIONS

In [ ]:
oxy = openmc.Material()
oxy.add_element('O', 1.0)
oxy

In [ ]:
oxy.add_nuclide('Am242_m1', 0.34)

In [ ]:
openmc.data.water_density

## Defining Geometry
  1. Surfaces -- definition of some surface equation
    $f(x,y,z) = x^2 + y^2 + z^2 - R^2 = 0$
  2. Regions -- well-defined volume composed of the intersection (&) and union (|) of surface half-spaces
  3. Cells -- assignment of material, temperature, etc. to a region

In [ ]:
sph = openmc.Sphere(r=10.0, boundary_type='vacuum')

## Surface operations: 

  - "positive" halfspace of the surface: **+**
  - "negative" halfspace of the surface: **-**

In [ ]:
inside_sphere = -sph

In [ ]:
inside_sphere

In [ ]:
(0, 0, 0) in inside_sphere

In [ ]:
(1, 0, 0) in inside_sphere

In [ ]:
(10, 0, 0) in inside_sphere

In [ ]:
openmc.Sphere

In [ ]:
plane = openmc.XPlane()

In [ ]:
+plane # above plane
-plane # below plane

## Region operators:

  - & (intersection)
  - | (union)
  - ~ (complement)

In [ ]:
# intersection operation
top_half_of_sphere = -sph & +plane

In [ ]:
(-1, 0, 0) in top_half_of_sphere

In [ ]:
(1, 0, 0) in top_half_of_sphere

In [ ]:
# union operation
some_region = -sph | -plane

In [ ]:
(10.1, 0, 0) in some_region

In [ ]:
# complement operator
opposite_some_region = ~some_region

In [ ]:
(-1, 0, 0) in some_region

In [ ]:
(-1, 0, 0) in opposite_some_region

In [ ]:
steel_cell = openmc.Cell(fill=steel, region=-sph)
air_cell = openmc.Cell(fill=air, region=+sph)

In [ ]:
air_cell

In [ ]:
steel_cell

In [ ]:
universe = openmc.Universe(name='my_universe', cells=[steel_cell, air_cell])

In [ ]:
universe

In [ ]:
universe.plot(width=(20.0, 20.0))

In [ ]:
inside_sphere.bounding_box

In [ ]:
outside_sphere = +sph

In [ ]:
outside_sphere.bounding_box

In [ ]:
top_half_of_sphere

## Running a Pincell Problem

In [ ]:
uo2 = openmc.Material()
uo2.add_nuclide('U235', 0.02)
uo2.add_nuclide('U238', 0.86)
uo2.add_element('O', 0.12)
uo2.set_density('g/cc', 10.0)

In [ ]:
uo2.volume = 10.0 # cm^3

In [ ]:
uo2.fissionable_mass

In [ ]:
zr = openmc.Material()
zr.add_element('Zr', 1.0)
zr.set_density('g/cc', 6.0)

In [ ]:
water = openmc.Material()
water.add_element('H', 2.0)
water.add_element('O', 1.0)
water.set_density('g/cc', 1.0)
water.add_s_alpha_beta('c_H_in_H2O')

In [ ]:
materials = openmc.Materials([uo2, zr, water])
materials.export_to_xml()

In [ ]:
fuel_outer_radius = openmc.ZCylinder(r=0.39)
clad_outer_radius = openmc.ZCylinder(r=0.46)

pitch = 1.26 # cm

left = openmc.XPlane(-pitch/2)
right = openmc.XPlane(pitch/2)
bottom = openmc.YPlane(-pitch/2)
top = openmc.YPlane(pitch/2)

inside_box = +left & -right & +bottom & -top

In [ ]:
openmc.rectangular_prism

In [ ]:
for surface in [left, right, bottom, top]:
    surface.boundary_type = 'reflective'

In [ ]:
fuel_cell = openmc.Cell(fill=uo2, region=-fuel_outer_radius)
clad_cell = openmc.Cell(fill=zr, region=+fuel_outer_radius & -clad_outer_radius)
water_cell = openmc.Cell(fill=water, region=+clad_outer_radius & inside_box)

In [ ]:
universe = openmc.Universe(cells=[fuel_cell, clad_cell, water_cell])
universe.plot(width=(pitch, pitch))

In [ ]:
geom = openmc.Geometry(root=universe)
geom.export_to_xml()

In [ ]:
!cat geometry.xml

In [ ]:
settings = openmc.Settings()
settings.batches = 20
settings.inactive = 10
settings.particles = 1000

In [ ]:
settings.export_to_xml()

In [ ]:
!cat settings.xml

In [ ]:
openmc.run()

In [ ]:
!ls

## The Model Class

In [ ]:
model = openmc.Model(geometry=geom, materials=materials, settings=settings)

In [ ]:
sp_filename = model.run()

In [ ]:
sp_filename

# Lattices

In [ ]:
cell_univ = model.geometry.root_universe
print(cell_univ.cells)

In [ ]:
print("Bounding box before {}".format(cell_univ.bounding_box))
# no outer planes on the outer cell region
water_cell.region = +clad_outer_radius
print("Bounding box after {}".format(cell_univ.bounding_box))

In [ ]:
lattice = openmc.RectLattice()
lattice.universes = 3 * [[cell_univ, cell_univ, cell_univ]]
lattice.pitch = (1.26, 1.26)
lattice.lower_left = (-3*0.63, -3*0.63)

In [ ]:
lattice_bounds = openmc.model.rectangular_prism(3 * 1.26, 3 * 1.26, boundary_type='reflective')

In [ ]:
lattice_cell = openmc.Cell(region=lattice_bounds, fill=lattice)
lattice_univ = openmc.Universe(cells=[lattice_cell])
geom = openmc.Geometry(root=lattice_univ)

In [ ]:
lattice_univ.plot(width=(4.0, 4.0), pixels=(800, 800))

In [ ]:
model.geometry = geom
model.run()